# 🧠 Stacking Sats Challenge: Overview

The Stacking Sats Challenge invites you to build a Bitcoin accumulation strategy that dynamically distributes a fixed budget across a four-year cycle to maximize performance, measured in Satoshis per Dollar (SPD) percentile.

---

## 🎯 Objective

Design a model that decides how much to buy each day over a four-year cycle using historical data as input. Your goal is to outperform the uniform DCA strategy in terms of SPD percentile, a metric that compares your model's performance to the best- and worst-case scenarios.

---

#### 📐 How SPD and SPD Percentile Are Calculated

**Satoshis per Dollar (SPD)** measures how many sats you get for every dollar spent:

$$
\text{spd} = \left(\frac{1}{\text{BTC/USD}}\right) \times 100{,}000{,}000
$$

> 📌 *Note: 100,000,000 sats = 1 BTC*

Because spd values can vary significantly over time, we convert them to a **spd percentile** score to better visualize how much value a uniform DCA strategy captures relative to the extreme cases:

$$
\text{spd\_pct} = \left(\frac{\text{strategy\_spd} - \text{worst\_spd}}{\text{best\_spd} - \text{worst\_spd}}\right) \times 100
$$

> ⚠️ *We normalize the total budget to 1 to make the model budget-invariant. This allows general comparison without concern for absolute dollar amounts or slippage (the latter is out of scope for this challenge).*

---

## 🔍 What You’re Building

You're creating a **feature-driven model** that takes in historical data (price, on-chain metrics, etc.) and outputs **daily buy weights**—a value between `MIN_WEIGHT` and 1. The weights determine how much of your budget to invest each day. The full budget must be used within each 4-year cycle.

---

## 📅 Challenge Setup

| Item                    | Description |
|-------------------------|-------------|
| **Backtest Period**     | `2013-01-01` to `2024-12-31` (3 full 4-year cycles) |
| **Accumulation Cycle**  | 4 years per cycle |
| **Total Budget**        | Normalized to 1 per cycle |
| **Data**                | Daily Bitcoin price and on-chain data from Coin Metrics (`2010-07-18` onward) |

---

## ✅ Model Validity Criteria

Your model must satisfy **all** of the following rules:

1. **Positive Daily Purchases**  
   Every day must have a strictly positive allocation: $\text{allocation}_t \geq \text{MIN\_WEIGHT} = 1 \times 10^{-5}$

2. **Budget Completeness**  
   The sum of all daily allocations must equal 1 for each cycle.

3. **No Forward-Looking Data**  
   The model must use only current and past data—no future leakage.

4. **Performance Requirement**  
   Your model’s SPD percentile must exceed **uniform DCA** on **all three cycles**.

---

## 🏁 Evaluation Criteria

Among valid models, winners are selected based on:

- **Highest SPD Percentile** averaged across the three 4-year cycles.

---

## 📦 Deliverables

1. **Model Code**  
   Submit your implementation using the provided template from **Tutorial 3** on [**Hypertrial.ai**](https://www.hypertrial.ai/)

2. **Explanation Notebook**  
   Create and submit a tutorial-style notebook describing your strategy to the [**Hypertrial GitHub repo**](https://github.com/hypertrial/Stacking-Sats/tree/main)

---

# Formal Mathematical Formulation (For the Quants)

Let $N$ be the number of trading days in one accumulation cycle (4 years), and let $p_i$ denote Bitcoin’s closing price on day $i$. Our total investment budget is normalized to 1 and allocated over these $N$ days using a vector of daily weights $\mathbf{w} = (w_1, w_2, \dots, w_N)$.

Each weight must satisfy the following:

$$
w_i \geq \text{MIN\_WEIGHT} = 10^{-5},\quad 
\sum_{i=1}^N w_i = 1.
$$

This guarantees a strictly positive allocation every day, avoiding zero-buys while respecting the full budget constraint.

---

### Uniform DCA (Baseline)

Uniform Dollar Cost Averaging corresponds to equal allocation on each day:

$$
w_i^{\text{uniform}} = \frac{1}{N},\quad \forall i \in \{1,\dots,N\}.
$$

This forms the baseline that your model must outperform.

---

### Dynamic-DCA: Model Definition and Optimization Objective

The objective of this challenge is to build a **data-driven model** that generates a valid allocation vector over the accumulation cycle and improves upon uniform DCA.

A valid model is a function:

$$
f(\text{features}) \mapsto \mathbf{w} \in \mathrm{int}\,\Delta^{N-1}
$$

where:
- **Features** are observable inputs derived from the available data (e.g., price moving average, volatility, on-chain indicators)
- $\mathbf{w}$ is a vector of daily allocation weights
- The output $\mathbf{w}$ must satisfy the following constraints:

$$
\mathrm{int}\,\Delta^{N-1} = \left\{ \mathbf{w} \in \mathbb{R}^N \,\middle|\, w_i \geq 10^{-5},\; \sum_{i=1}^N w_i = 1 \right\}
$$

This is the (clipped) probability simplex, which ensures:

- **Strictly positive daily purchases**: $w_i \geq 10^{-5}$  
- **Full budget utilization**: $\sum_i w_i = 1$

The model must use **only current and past data**—no future information is allowed. This ensures the strategy is deployable in real time and avoids overfitting.

---

### Objective: Maximize Sats per Dollar (SPD)

The performance metric is **sats-per-dollar (SPD)**, defined as:

$$
\mathrm{SPD}(\mathbf{w}) = \sum_{i=1}^N w_i \cdot \left( \frac{1}{p_i} \right)
$$

Here:
- $p_i$ is the closing BTC price on day $i$
- $\frac{1}{p_i}$ gives BTC per dollar
- $w_i$ scales the sats acquired by that day's budget share

Your goal is to build a model that outputs weights $\mathbf{w}$ which maximize SPD:

$$
\max_{f} \;\mathrm{SPD}(f(\text{features}))
$$

That is, based on a set of daily features (BTC price, on-chain metrics), your model must output a valid weight vector $\mathbf{w}$ that determines the buying schedule.

> **Note: Why Maximizing SPD ≡ Maximizing SPD Percentile**
>
> The SPD percentile is calculated as:
>
> $$
\text{spd\_pct} = \left(\frac{\text{strategy\_spd} - \text{worst\_spd}}{\text{best\_spd} - \text{worst\_spd}}\right) \times 100
$$
>
> where:
> - `best_spd` = buying 100% at the cycle low  
> - `worst_spd` = buying 100% at the cycle high  
> - Both values are **fixed** for a given cycle
>
> Because the denominator is constant for each cycle, and your strategy only affects `strategy_spd`, **maximizing SPD will also maximize SPD percentile**.

---

### Not Just Optimization — A Valid Predictive Model

This challenge is **not** about cherry-picking ideal weights using future prices.

> You must build a model that generalizes from observable features to weight allocations **without access to future data**.

In short, you’re solving a constrained optimization problem **indirectly**, through a model that maps features to valid decisions in a deployable way.

---

### Summary

Your model should:
- Take in **observable features** (at or before time $i$)
- Produce **strictly positive, normalized daily weights**
- Operate **causally**, without future data
- **Outperform uniform DCA** across backtested cycles
- Be evaluated using **SPD** (or its percentile form)

This preserves the benefits of DCA—discipline, regularity, and emotional neutrality—while introducing dynamic buys to capture more sats per dollar.


## Geometry of the Optimization

Because SPD is a **linear function** of the allocation vector $\mathbf{w}$, its unconstrained maximum over the **closed** simplex

$$
\Delta^{N-1} = \left\{ \mathbf{w} \in \mathbb{R}^N \;\middle|\; w_i \geq 0,\; \sum_{i=1}^N w_i = 1 \right\}
$$

occurs at a **vertex**—in practice, this means allocating 100% of the budget to the single day with the lowest price.

To prevent such corner (all-in) solutions, we constrain the weights to the **interior** of the simplex:

$$
\mathrm{int}\,\Delta^{N-1} = \left\{ \mathbf{w} \in \mathbb{R}^N \;\middle|\; w_i \geq 10^{-5},\; \sum_{i=1}^N w_i = 1 \right\}
$$

This introduces a necessary tradeoff:

- **Maximizing SPD** favors leaning into lower-priced days  
- **Maintaining DCA principles** enforces a more balanced schedule (closer to uniform)

### Why constrain to the clipped (open) simplex?

- **Prevents degenerate, overfit allocations.**  
  A linear objective pushes toward extremes—forcing $w_i \geq 10^{-5}$ ensures diversified daily purchases.

- **Preserves DCA’s discipline.**  
  A strictly positive allocation guarantees that every day receives some investment.

- **Maintains convexity.**  
  The feasible region is convex: any average of two valid solutions is still a valid solution. This improves robustness and interpretability of the optimization landscape.

---

## Deeper Dive: Feature-Driven Rules for Dynamic DCA

In this framework, a valid strategy is defined as a deterministic mapping:

$$
f: \mathcal{X} \rightarrow \mathrm{int}\,\Delta^{N-1}
$$

where:
- $\mathcal{X}$ is the space of observable features (e.g., rolling volatility, momentum, price drawdown)
- $\mathrm{int}\,\Delta^{N-1}$ is the clipped simplex of valid weight vectors

At each cycle $t$, the model receives a feature vector $\mathbf{x}^{(t)}$ and outputs a valid allocation vector:

$$
\mathbf{w}^{(t)} = f\left( \mathbf{x}^{(t)}; \theta \right)
$$

with the constraints:

$$
w_i^{(t)} \geq 10^{-5},\quad 
\sum_{i=1}^N w_i^{(t)} = 1
$$

### Why formalize models this way?

1. **Avoids hindsight bias.**  
   Allocations are **functionally generated**, not manually selected after seeing prices.

2. **Supports generalization.**  
   By choosing a parameterized function class (e.g., linear model, decision tree, neural net), models can be validated and tuned properly on historical cycles.

3. **Allows reproducibility and comparison.**  
   All strategies live in the same feasible space. Performance differences reflect model and feature quality—not optimization shortcuts.

4. **Encourages structural insight.**  
   Successful models reveal patterns and structural dynamics in Bitcoin’s behavior that improve DCA performance beyond uniform allocation.

